In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ['EIN', 'NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(counts[counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values 

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =78)

C:\Users\ashle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

   # Attempt one at 75% Accuracy - Added additional hidden layers


In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input = len(X_train[0])
hidden_layer1 = 86
hidden_layer2 = 50
hidden_layer3 = 30
hidden_layer4 = 44

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation='relu', input_dim = number_input))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))

# Third and fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 86)                3784      
                                                                 
 dense_1 (Dense)             (None, 50)                4350      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 44)                1364      
                                                                 
 dense_4 (Dense)             (None, 1)                 45        
                                                                 
Total params: 11,073
Trainable params: 11,073
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [15]:
# Train the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


cp_callback = ModelCheckpoint(filepath=checkpoint_path,verbose=1,
    save_weights_only=True,save_freq=4020)

fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

nn.save('AlphabetSoupCharity_Optimization.h5')

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 13305.1621 - accuracy: 0.4884
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 2174.9561 - accuracy: 0.5010
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 4021.4290 - accuracy: 0.4963
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 1489.1125 - accuracy: 0.5019
Epoch 5/100
800/804 [============================>.] - ETA: 0s - loss: 2090.1968 - accuracy: 0.5039
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 2080.1277 - accuracy: 0.5036
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 1.5833 - accuracy: 0.5320
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/100
804/804 [==============

804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 65/100
794/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5317
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 69/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 70/100
800/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5323
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 71/100
804/804 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7065 - accuracy: 0.5306 - 1s/epoch - 5ms/step
Loss: 0.7065237760543823, Accuracy: 0.5306122303009033


   # Attempt 2 at 75% accuracy: 2 hidden layers, adjusted neurons, epochs 50

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input = len(X_train[0])
hidden_layer1 = 129
hidden_layer2 = 40

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer1, activation='relu', input_dim = number_input))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 129)               5676      
                                                                 
 dense_6 (Dense)             (None, 40)                5200      
                                                                 
 dense_7 (Dense)             (None, 1)                 41        
                                                                 
Total params: 10,917
Trainable params: 10,917
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [19]:
# Train the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


cp_callback2 = ModelCheckpoint(filepath=checkpoint_path,verbose=1,
    save_weights_only=True,save_freq=4020)

fit_model2 = nn2.fit(X_train, y_train, epochs=50, callbacks=[cp_callback2])

nn2.save('AlphabetSoupCharity_Optimization.h5')

Epoch 1/50
804/804 [==============================] - 6s 5ms/step - loss: 52725.0625 - accuracy: 0.4976
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 38215.6133 - accuracy: 0.4904
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 30935.7949 - accuracy: 0.5066
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 24638.6914 - accuracy: 0.4931
Epoch 5/50
798/804 [============================>.] - ETA: 0s - loss: 28368.2051 - accuracy: 0.5127
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 28165.8535 - accuracy: 0.5128
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 4462.9912 - accuracy: 0.5122
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 22432.6543 - accuracy: 0.5037
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 23379.9375 - accuracy: 0.5106
Epoch 9/50
804/804 [=======

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8115 - accuracy: 0.5359 - 1s/epoch - 4ms/step
Loss: 0.8115087747573853, Accuracy: 0.5358600616455078


   # Attempt 3 at 75% accuracy: using different activation functions

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input = len(X_train[0])
hidden_layer1 = 80
hidden_layer2 = 30
hidden_layer3 = 20 

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer1, activation='sigmoid', input_dim = number_input))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer2, activation='sigmoid'))

# Third hidden layer 
nn3.add(tf.keras.layers.Dense(units=hidden_layer2, activation='sigmoid'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 30)                2430      
                                                                 
 dense_10 (Dense)            (None, 30)                930       
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,911
Trainable params: 6,911
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn3.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [23]:
# Train the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


cp_callback2 = ModelCheckpoint(filepath=checkpoint_path,verbose=1,
    save_weights_only=True,save_freq=4020)

fit_model2 = nn2.fit(X_train, y_train, epochs=50, callbacks=[cp_callback2])

nn2.save('AlphabetSoupCharity_Optimization.h5')

Epoch 1/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 5/50
796/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5318
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 8/50
804/804 [==============================] - 4s 6ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 9/50
804/804 [==============================] - 4s 5ms/s

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6922 - accuracy: 0.5332 - 1s/epoch - 5ms/step
Loss: 0.6922488212585449, Accuracy: 0.5331778526306152
